# Loading Images with GEE

In [16]:
import ee
import geemap

ee.Authenticate()
ee.Initialize(project='karlzam')

In [34]:
Map = geemap.Map()

# Large scale international boundary polygons, simplified
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

# Using swaziland as an example because it's small
roi = countries.filter(ee.Filter.eq("country_na", "Canada"));

# Get map
Map.addLayer(roi, {}, "Canada")

# Center the object at the region of interest with a zoom of 8
Map.centerObject(roi,4)

# Load in landsat data with filter
# Originally this covers the entire world over a long time series 
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1").filterDate('2023-09-22', '2023-10-24').filterBounds(roi)

# Remove clouds
composite = ee.Algorithms.Landsat.simpleComposite(**{
    'collection': landsat,
    'asFloat': True
})

# Define landsat bands we'll be using 
rgbVis = {'bands': ['B4', 'B3', 'B2'], 'min':0, 'max':0.3}

Map.addLayer(composite.clip(roi), rgbVis, "Landsat RGB")

# Launch map
Map

Map(center=[60.55843820087575, -96.70702570923034], controls=(WidgetControl(options=['position', 'transparent_…

In [35]:
# This adds FIRMS fires to the map from the time range we're interested in
burned_areas = ee.ImageCollection("FIRMS").filterDate('2023-09-22', '2023-09-24')
fires = burned_areas.select('T21')

firesVis = {'min': 0, 'max': 500.0, 'palette': ['red', 'orange', 'yellow']}

Map.addLayer(fires, firesVis, "FIRMS")